#### Import modules

In [1]:
import numpy as np #matrix  manipulation
import os #filesystem support
import pandas as pd #dataframe 
import Levenshtein #calculating edit distance

#### Set project folders

In [2]:
PROJECT_FOLDER = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
RAW_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'raw')
PROCESSED_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'processed')
FINAL_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'final')

#### Load dataframe

In [3]:
DATA =  os.path.join(PROCESSED_DATA_FOLDER, 'benefit_data_processed.feather')
df = pd.read_feather(DATA)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3094 entries, 0 to 3093
Columns: 189 entries, workerid to treatment_str
dtypes: float64(87), int64(5), int8(1), object(96)
memory usage: 4.4+ MB


#### Make treatment string for each main effect 

In [4]:
def main_treatment(row):
    treatment_pair = row['treatment_str'].split(' + ')
    return pd.Series(treatment_pair)

treatment_columns = ['treatment_first', 'treatment_second', 'treatment_third']
df[treatment_columns]  = df.apply(lambda row: main_treatment(row), axis=1)
df.treatment_third.fillna('No Clarification', inplace=True)

#### Make indicator variables for each main effect

In [5]:
dummies = pd.get_dummies(df[treatment_columns], prefix='', prefix_sep='', drop_first=False)
dummies.rename(lambda x: x.lower().replace(' ', '_'), axis=1, inplace=True)
df = df.merge(dummies, left_index=True, right_index=True)

#### Convert timeseries columns

In [6]:
timseries = ['Start Date', 'End Date']
for series in timseries:
    df[series] = pd.to_datetime(df[series], infer_datetime_format=True)

df['duration_survey'] = df['End Date'] - df['Start Date']
df['duration_survey'] = df['duration_survey'].dt.seconds
df['duration_work'] = df['end_time'] - df['start_time']
df = df.drop(df.columns[[1, 2, 183, 184]], axis=1)

#### Type convert string variables to integer variables

In [7]:
genders = {"Male": 0, "Female": 1}
df["female"] = df["female"].replace(genders)
device = {"No": 0, "Yes": 1}
df["mobile_device"] = df["mobile_device"].replace(device)

#### Type convert categorial variables to integer variables

In [8]:
categorial = ['enjoyment', 'motivation', 'future_motivation', 'latin', 'touch_typing']
for cats in categorial:
    df[cats] = df[cats].astype('category').cat.codes + 1

#### Replace, reorder and type convert education variable

In [9]:
to_replace = {'Professional Degree (JD, MD)':'Doctoral Degree'}
df['education'] = df['education'].replace(to_replace)
df['education'] = df['education'].astype('category')

education_ordered = pd.Index([
    u'High School / GED',
    u'Some College',
    u'2-year College Degree', 
    u'4-year College Degree', 
    u'Masters Degree',
    u'Doctoral Degree', 
    ], dtype='object')   

df.education.cat.set_categories(education_ordered, inplace=True)
df['education'] = df['education'].cat.codes + 1

#### Remove leading/trailing whitespace and linebreaks

In [10]:
def remove_whitespace(x):
    if isinstance(x, str):
        return x.strip().replace('\n', ' ').replace('\r', '')
    else:
        return x

df = df.applymap(remove_whitespace)

#### Make columns with correct solutions

In [11]:
FRAGMENTS = os.path.join(RAW_DATA_FOLDER, 'fragments.xlsx')

df_fragments = pd.read_excel(FRAGMENTS, header=None, names=["Keyword", "Solution"], encode='utf8')\
                 .applymap(lambda x: x.strip())\
                 .drop('Keyword', axis=1)

for index, row in df_fragments.iterrows():
    column = 'question{0}_solution'.format(index + 1)
    df[column] = row['Solution']

#### Make dataframe with empty columns for edit distance and error_share, then join with main dataframe

In [12]:
empty_cols = ['question{0}_distance', 'question{0}_editratio']
edit_columns = [elem.format(x) for x in range(1, 81) for elem in empty_cols]
edit_frame = pd.DataFrame(index=df.index, columns=edit_columns)
df = df.merge(edit_frame, left_index=True, right_index=True)

#### Calculate Levenshtein edit distance 

In [13]:
answer_columns = ['question{0}_answer'.format(x) for x in range(1, 81)]
solution_columns = ['question{0}_solution'.format(x) for x in range(1, 81)]
column_pairs = list(zip(answer_columns, solution_columns))

for index, row in df.iterrows():

    number_answers_submitted = row.at['counter']
    columns_to_check = column_pairs[:number_answers_submitted]

    for number, (answer_col, solution_col) in enumerate(columns_to_check):
        answer = row[answer_col]
        solution = row[solution_col]

        if isinstance(answer, str):
            edit_distance = Levenshtein.distance(answer, solution)
            edit_ratio = (edit_distance - answer.count('?')) / max(len(answer), len(solution))

        else:
            edit_distance = np.NAN
            edit_ratio = np.NAN

        dist_column = 'question{0}_distance'.format(number + 1)
        ratio_column = 'question{0}_editratio'.format(number + 1)

        df.loc[index, dist_column] = edit_distance
        df.loc[index, ratio_column] = edit_ratio

#### Count non-empty answers

In [14]:
def count_non_empty(row):
    raw_submits = row['counter']
    answer_columns = ['question{0}_answer'.format(x) for x in range(1, raw_submits + 1)]
    return row[answer_columns].count()

df['counter_real'] = df.apply(count_non_empty, axis=1)
df['counter_empty'] = df['counter'] - df['counter_real']
df['counter_quota'] =  df['counter_empty'] / df['counter']

#### Order dataframe columns

In [15]:
cols = [
    'question{0}_answer',
    'question{0}_solution',
    'question{0}_distance',
    'question{0}_editratio',
    'question{0}_timer',
]

ques_cols = [elem.format(x) for x in range(1,81) for elem in cols]
main = list(df.columns.difference(ques_cols + dummies.columns.tolist()))
main = main + dummies.columns.tolist()
df = df.reindex(main + ques_cols, axis=1)    

#### Save dataframe

In [16]:
path = os.path.join(FINAL_DATA_FOLDER, 'benefit_data.feather')
df.to_feather(path)